In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
# import required libraries
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
import nltk
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow import metrics
from tensorflow.keras.models import load_model

In [3]:
import pickle
# import a naive bayes
NaiveBayes_model = pickle.load(
    open(
        "/content/gdrive/My Drive/NLP Project/NB_model.pkl",
        "rb",
    )
)

# import the count vectorizer for making Bag of Words
Count_Vectorizer = pickle.load(
    open(
        "/content/gdrive/My Drive/NLP Project/CV_BOG.pkl",
        "rb",
    )
)

# import a Tokenizer for assigning a unique integer to one
Tokenizer1 =  pickle.load(
    open(
        "/content/gdrive/My Drive/NLP Project/Text_Vec.pkl",
        "rb",
    )
)

In [4]:
# import neural network


BiLSTM_Model = load_model(
    "/content/gdrive/My Drive/NLP Project/neural_network.h5"
)

In [5]:
# Loading the dataset
fake_news = pd.read_csv("/content/gdrive/My Drive/NLP Project/Fake_news_data.csv")

In [6]:
fake_news_test = fake_news.iloc[14999:,:] # get the last 5808 rows a test data completely unseen

In [7]:
fake_news_test = fake_news_test.dropna()  # place this above the two cells
label = fake_news_test["label"]  # Get a seperate columns for labels
news_test = fake_news_test.drop("label", axis=1)
news_test.set_index("id", inplace=True)
                                # all these operations are performed on test data chosen 
                                # above in the cell

news_test["text"] = news_test[
    "text"
].str.lower()  # convert the whole text to lower case to ensure uniformity

news_test["text"] = news_test["text"].replace("[^A-Za-z\s]", "", regex=True)
#Replace everything that is not letters or a space with a blank

news_test["text"] = news_test["text"].str.split()  # split our text column to a list

In [8]:
ps = PorterStemmer()
nltk.download("stopwords")

# make a stemmer and remove stopwords
def remove_stopwords_and_stem(x):
    stopwds_lst = stopwords.words("english")
    sentence = ""
    for i in x:
        if i in stopwds_lst:
            x.remove(i)
    for k in range(len(x)):
        word = ps.stem(x[k])
        x[k] = word
    for j in x:
        sentence = sentence + j + " "
    sentence_final = sentence[:-1]
    return sentence_final

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
news_test['text'] = news_test['text'].apply(remove_stopwords_and_stem) # apply the function

In [10]:
text_comments = news_test['text'] 
# now we seperate the text into another series 

In [11]:
BOG_test = Count_Vectorizer.transform(text_comments) # We pass test input to transform into BOG, 
                                                    # CV is fitted on train and transformed on test
BOG_test = BOG_test.toarray() # seperate BOG(bag of words as a seperate array)
label = np.array(label) # seperate labels as an array

In [12]:
predicted = NaiveBayes_model.predict(BOG_test)
from sklearn import metrics
metrics.accuracy_score(label, predicted) # make prediction with Naivebayes
metrics.recall_score(label, predicted) # make prediction with Naivebayes

0.8645973909131804

In [13]:
text_comments = list(news_test['text']) # make a list of all documents in the data
one_hot_encoded = Tokenizer1.texts_to_sequences(text_comments)
padded_vector = pad_sequences(one_hot_encoded, padding = 'post', maxlen= 500)
input =np.array(padded_vector)
output = np.array(label)  

In [14]:
BiLSTM_Model.evaluate(input, output)

160/160 [==============================] - 39s 234ms/step - loss: 0.2532 - accuracy: 0.9338 - recall: 0.9186


[0.2531854212284088, 0.9337514638900757, 0.9185785055160522]

In [15]:
predicted_prob = BiLSTM_Model.predict(input, batch_size=64)

In [16]:
keras_predicted = np.where(predicted_prob > 0.5, 1,0)

In [17]:
  print(classification_report(label, predicted))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91      2879
           1       0.90      0.86      0.88      2223

    accuracy                           0.90      5102
   macro avg       0.90      0.89      0.90      5102
weighted avg       0.90      0.90      0.90      5102



In [18]:
print(classification_report(label, keras_predicted))


              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2879
           1       0.93      0.92      0.92      2223

    accuracy                           0.93      5102
   macro avg       0.93      0.93      0.93      5102
weighted avg       0.93      0.93      0.93      5102

